# **HW1: Regression**

In _assignment 1_, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases

> - Step 1: Split Data
> - Step 2: Preprocess Data
> - Step 3: Implement Regression
> - Step 4: Make Prediction
> - Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part


# 1. Basic Part (60%)

In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random


## Global attributes

Define the global attributes


In [ ]:
# Input file named as 'hw1_basic_input.csv'
input_dataroot = 'hw1_basic_input.csv'
# Output file will be named as 'hw1_basic.csv'
output_dataroot = 'hw1_basic.csv'

# Initial datalist, saved as numpy array
input_datalist = []
# Your prediction, should be 10 * 4 matrix and saved as numpy array
output_datalist = []
# The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

train_dataset = []
validation_dataset = []
toPredict_dataset = []
models = []


You can add your own global attributes here


In [ ]:
MatrixInversion = "MatrixInversion"
GradientDescent = "GradientDescent"

# data
input_ratio = 1
output_weeks = 10
train_all = False
validation_ratio = 0.2
preprocess_data = True
preprocess_all_columns = True
drop_unwanted = False

# auto regression
previous_weeks = 5
order_per_weeks = 4
# fitting_method = MatrixInversion
fitting_method = GradientDescent

# gradient descent
learning_rate = 5e-2
iterations = 50000

# pd.set_option('display.max_rows', df.shape[0]+1)


In [ ]:
def calc_mape(pred_Y, Y):
    mape = np.mean(np.absolute((Y - pred_Y) / Y), dtype=np.float32) * 100
    return mape


def get_xy(input_data, x_idx):
    ret_x = []
    for i in range(3):
        X = input_data[previous_weeks:, [i+ii for ii in x_idx]]
        if (previous_weeks):
            for d in range(previous_weeks):
                for o in range(order_per_weeks):
                    X = np.hstack((X, np.float_power(input_data[d:-previous_weeks+d, [i+4]], o+1)))
        ret_x.append(X)
    ret_y = [input_data[previous_weeks:, [i+4]] for i in range(3)]
    return (ret_x, ret_y)


class auto_regression_model:
    def __init__(self, method=MatrixInversion):
        self.method = method
        self.w = None

    def fit(self, X, Y):
        m, n = X.shape
        self.M = X.mean(axis=0)
        self.S = X.max(axis=0) - X.min(axis=0)
        X = self.normalize(X)
        X = np.hstack((np.ones((m, 1)), X))

        if (self.method == MatrixInversion):
            ''' w = ((phi^T)*(phi))^-1 * phi^T * y '''
            self.w = np.linalg.lstsq(X, Y, rcond=None)[0]

        elif (self.method == GradientDescent):
            pred_Y = np.copy(Y)
            # for it in range(10):
            for it in range(iterations):
                if (it == 0):
                    self.w = np.random.rand(n+1,1)
                pred_Y = X.dot(self.w)
                diff_y = Y - pred_Y
                g = 2 * np.mean(diff_y * X, axis=0).reshape(-1, 1)
                self.w = self.w + learning_rate * g

    def predict(self, X: np.ndarray):
        m, n = X.shape
        X = X.copy()
        pred_Y = np.ndarray((m, 1))
        for r in range(m):
            row_X = X[[r]]
            yy = self.get_y(row_X)[0, 0]
            pred_Y[[r], 0] = yy
            for d in range(previous_weeks, 0, -1):
                if (r + d >= m):
                    break
                # X[r+d, -d] = yy
                for o in range(order_per_weeks):
                    X[r+d, -d*order_per_weeks+o] = yy
            # print(X.astype(np.int32))
        return pred_Y

    def get_y(self, X: np.ndarray):
        X = self.normalize(X)
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return X.dot(self.w)

    def normalize(self, X: np.ndarray):
        return (X - self.M) / self.S

    def get_model(self):
        return self.w


models.append(auto_regression_model(method=fitting_method))

models.append(auto_regression_model(method=fitting_method))

models.append(auto_regression_model(method=fitting_method))


## Load the Input File

First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in _input_datalist_


In [ ]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
    input_datalist = np.array(list(csv.reader(csvfile)))


## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 0: Preprocess


In [ ]:
def PreprocessDatalist(datalist, x_idx=[1, 2, 3]):
    df = pd.DataFrame(datalist)
    df = df.iloc[1:]
    df = df.replace('', np.nan).astype(np.float32)

    toPredict = df.iloc[-output_weeks:].to_numpy()
    df = df.iloc[:-output_weeks]

    if preprocess_data:
        Q1 = df.quantile(0.25)
        Q3 = df.quantile(0.75)
        IQR = Q3 - Q1
        ldf = df < (Q1 - 1.5 * IQR)
        rdf = df > (Q3 + 1.5 * IQR)
        if (preprocess_all_columns):
            df[(ldf | rdf)] = np.nan

        else:
            df[(ldf | rdf).loc[:, x_idx]] = np.nan

    if(drop_unwanted):
        df = df.dropna()
    else:
        df = df.interpolate()
        df = df.fillna(method="ffill")
        df = df.fillna(method="bfill")

    datalist = df.to_numpy()
    if (previous_weeks > 0):
        toPredict = np.vstack((datalist[-previous_weeks:], toPredict))

    # with open("datalist.csv", 'w', newline='', encoding="utf-8") as csvfile:
    #     writer = csv.writer(csvfile)
    #     for row in datalist:
    #         writer.writerow(row)

    return (datalist, toPredict)

### Step 1: Split Data

Split data in _input_datalist_ into training dataset and validation dataset


In [ ]:
def SplitData(datalist):
    print("Spliting Data...")

    # print(pd.DataFrame(datalist).describe())
    input_weeks = int(datalist.shape[0] * input_ratio)
    datalist = datalist[-input_weeks:]
    # print(pd.DataFrame(datalist).describe())
    print(f"data weeks: {input_weeks}")

    train_weeks = int(datalist.shape[0] * (1 - validation_ratio))
    train = []
    if(train_all):
        train = datalist
    else:
        train = datalist[:train_weeks]
    validation = datalist[train_weeks-previous_weeks:]

    return (train, validation)


### Step 2: Preprocess Data

Handle the unreasonable data

> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics


In [ ]:
def PreprocessData():
    return


### Step 3: Implement Regression

> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part


In [ ]:
def Regression(models, train, validation, x_idx:list=[1]):
    train_x, train_y = get_xy(train, x_idx)
    validation_x, validation_y = get_xy(validation, x_idx)
    
    # train_x = []
    # for i in range(3):
    #     X = train[previous_weeks:, [i+ii for ii in x_idx]]
    #     if (previous_weeks):
    #         for d in range(previous_weeks):
    #             for o in range(order_per_weeks):
    #                 X = np.hstack((X, np.float_power(train[d:-previous_weeks+d, [i+4]], o+1)))
    #     train_x.append(X)
    # train_y = [train[previous_weeks:, [i+4]] for i in range(3)]
    # validation_x = []
    # for i in range(3):
    #     X = validation[previous_weeks:, [i+ii for ii in x_idx]]
    #     if (previous_weeks):
    #         for d in range(previous_weeks):
    #             for o in range(order_per_weeks):
    #                 X = np.hstack(
    #                     (X, np.float_power(validation[d:-previous_weeks+d, [i+4]], o+1)))
    #     validation_x.append(X)
    # validation_y = [validation[previous_weeks:, [i+4]] for i in range(3)]

    mape_sum = 0
    for i in range(len(models)):
        X, Y = train_x[i], train_y[i]
        models[i].fit(X, Y)

        if (validation_ratio != 0):
            X, Y = validation_x[i], validation_y[i]
            pred_y = models[i].predict(X)
            mape = calc_mape(pred_y, Y)
            mape_sum += mape
            print(f"MAPE {chr(i+65)}: {mape}%")

    for i in range(len(models)):
        plt.subplot(231)
        plt.title("train: y truth")
        plt.plot(train_y[i])
        plt.subplot(232)
        plt.title("train: x dot w")
        plt.plot(models[i].get_y(train_x[i]))
        plt.subplot(233)
        plt.title("train: predict(x)")
        plt.plot(models[i].predict(train_x[i]))
        plt.subplot(234)
        plt.title("validation: y truth")
        plt.plot(validation_y[i])
        plt.subplot(235)
        plt.title("validation: x dot w")
        plt.plot(models[i].get_y(validation_x[i]))
        plt.subplot(236)
        plt.title("validation: predict(x)")
        plt.plot(models[i].predict(validation_x[i]))
        plt.show()

    if (validation_ratio != 0):
        print(f"Average MAPE: {mape_sum / 3}%")
    return


### Step 4: Make Prediction

Make prediction of testing dataset and store the value in _output_datalist_


In [ ]:
def MakePrediction(models, toPredict, x_idx:list=[1]):
    def f(result):
        result[result < 0] = 0
        result = np.round(result).astype(np.int64)
        return result

    toPredict_x, toPredict_y = get_xy(toPredict, x_idx)

    # toPredict_x = []
    # for i in range(3):
    #     X = toPredict[previous_weeks:, [i+ii for ii in x_idx]]
    #     if (previous_weeks):
    #         for d in range(previous_weeks):
    #             for o in range(order_per_weeks):
    #                 X = np.hstack((X, np.float_power(toPredict[d:-previous_weeks+d, [i+4]], o+1)))
    #     toPredict_x.append(X)
    # toPredict_y = [toPredict[previous_weeks:, [i+4]] for i in range(3)]

    output_datalist = toPredict[-output_weeks:, [0]].astype(np.int64)

    for i in range(len(models)):
        result = f(models[i].predict(toPredict_x[i]))
        output_datalist = np.hstack((output_datalist, result))

    return output_datalist


### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted

- If your regression model is _3x^2 + 2x^1 + 1_, your output would be:

```
3 2 1
```


In [ ]:
input_datalist, toPredict_dataset = PreprocessDatalist(input_datalist)
# print(input_datalist.astype(np.int64))
train_dataset, validation_dataset = SplitData(input_datalist)
PreprocessData()
Regression(models, train_dataset, validation_dataset)
output_datalist = MakePrediction(models, toPredict_dataset)
print("validation ratio:", validation_ratio)
print("previous weeks:", previous_weeks)
print("orders per weeks:", order_per_weeks)

for i in range(3):
    print(f"model {chr(i+65)}:", models[i].get_model().reshape(-1))
    # print(models[i].M)
    # print(models[i].S)


## Write the Output File

Write the prediction to output csv

> Format: 'epiweek', 'CityA', 'CityB', 'CityC'


In [ ]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in output_datalist:
        writer.writerow(row)


# 2. Advanced Part (35%)

In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv**


## Attributes


In [ ]:
advanced_input_dataroot = 'hw1_basic_input.csv'
advanced_input1_dataroot = 'hw1_advanced_input1.csv'
advanced_input2_dataroot = 'hw1_advanced_input2.csv'
advanced_output_dataroot = 'hw1_advanced.csv'

advanced_input_datalist = []
advanced_input1_datalist = []
advanced_input2_datalist = []
advanced_output_datalist = []

train_dataset = []
validation_dataset = []
toPredict_dataset = []

models = [auto_regression_model(method=fitting_method) for _ in range(3)]


## Load Files


In [ ]:
# Read input csv to datalist
with open(advanced_input_dataroot, newline='') as csvfile:
    advanced_input_datalist = np.array(list(csv.reader(csvfile)))

with open(advanced_input1_dataroot, newline='') as csvfile:
    advanced_input1_datalist = np.array(list(csv.reader(csvfile)))

with open(advanced_input2_dataroot, newline='') as csvfile:
    advanced_input2_datalist = np.array(list(csv.reader(csvfile)))


### Implementation

In [ ]:
advanced_input_datalist = np.hstack(
    (advanced_input_datalist, advanced_input1_datalist[:, 1:]))
advanced_input_datalist, toPredict_dataset = PreprocessDatalist(
    advanced_input_datalist, x_idx=[1,2,3,7,8,9])
train_dataset, validation_dataset = SplitData(advanced_input_datalist)
Regression(models, train_dataset, validation_dataset, x_idx=[1,7])
advanced_output_datalist = MakePrediction(models, toPredict_dataset, x_idx=[1,7])

print("model A:", models[0].get_model().reshape(-1))
print("model B:", models[1].get_model().reshape(-1))
print("model C:", models[2].get_model().reshape(-1))


In [ ]:
# import torch
# import torch.nn as nn
# from torchmetrics.functional import mean_absolute_percentage_error

# train_x = []
# for i in range(3):
#     X = train_dataset[previous_weeks:, [i+1, i+7]]
#     if (previous_weeks):
#         for d in range(previous_weeks):
#             X = np.hstack((X, train_dataset[d:-previous_weeks+d, [i+4]]))
#     train_x.append(torch.from_numpy(X))
# train_y = [torch.from_numpy(train_dataset[previous_weeks:, [i+4]])
#            for i in range(3)]
# validation_x = []
# for i in range(3):
#     X = validation_dataset[previous_weeks:, [i+1, i+7]]
#     if (previous_weeks):
#         for d in range(previous_weeks):
#             X = np.hstack((X, validation_dataset[d:-previous_weeks+d, [i+4]]))
#     validation_x.append(torch.from_numpy(X))
# validation_y = [torch.from_numpy(
#     validation_dataset[previous_weeks:, [i+4]]) for i in range(3)]

# n_samples, n_features = train_x[0].shape


# class LinearRegression(nn.Module):

#     def __init__(self, input_dim, output_dim):
#         super(LinearRegression, self).__init__()

#         # define layers
#         self.linear = nn.Linear(input_dim, output_dim)

#     def forward(self, x):

#         return self.linear(x)


# models = [LinearRegression(n_features, 1) for _ in range(3)]

# learning_rate = 0.00001
# criterion = nn.MSELoss()
# epochs = 10000
# mape_sum = 0
# for i in range(len(models)):
#     print(f"Model {chr(i+65)} training.")
#     optimizer = torch.optim.SGD(models[i].parameters(), lr=learning_rate)
#     for epoch in range(epochs):
#         # forward pass and loss
#         pred_y = models[i](train_x[i])
#         # print(pred_y)
#         loss = criterion(pred_y, train_y[i])

#         # backward pass
#         loss.backward()

#         # update
#         optimizer.step()

#         # init optimizer
#         optimizer.zero_grad()

#         if (epoch + 1) % 10 == 0:
#             print(f'epoch: {epoch+1}, loss = {loss.item(): .4f}', end='\r')
#     print()

#     if(validation_ratio):
#         pred_y = [models[i](x) for x in validation_x[i]]
#         pred_y = torch.tensor(pred_y).reshape(-1,1)
#         mape = mean_absolute_percentage_error(pred_y, validation_y[i]) * 100
#         mape_sum += mape
#         print(f'MAPE = {mape}%')
# print(f'Average MAPE = {mape_sum / 3}%')


## Write Ouptut File


In [ ]:
with open(advanced_output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in advanced_output_datalist:
        writer.writerow(row)


# Report _(5%)_

Report should be submitted as a pdf file **hw1_report.pdf**

- Briefly describe the difficulty you encountered
- Summarize your work and your reflections
- No more than one page


# Save the Code File

Please save your code and submit it as an ipynb file! (**hw1.ipynb**)
